In [1]:
import os
import csv
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [4]:
from keras.models import Sequential, Model
from keras.layers import Flatten, Dense, Lambda, Convolution2D, Cropping2D
from keras.layers.pooling import MaxPooling2D

Using TensorFlow backend.


ssh ubuntu@insights-gpu01.escm.co

scp /Users/wding/Documents/work/BASF/ ubuntu@insights-gpu01.escm.co:/ml/emmading/

scp ~/Desktop/data.zip ubuntu@insights-gpu01.escm.co:/ml/emmading/project/
scp ~/Desktop/model_no_generator.py ubuntu@insights-gpu01.escm.co:/ml/emmading/project/
scp ubuntu@insights-gpu01.escm.co:/ml/emmading/project/model.h5 ~/Desktop/
    
KERAS_BACKEND=tensorflow python3 model_no_generator.py 

In [40]:
for i in os.listdir("CarND-Behavioral-Cloning-P3/run/"):
    if not i.endswith(".jpg"):
        print(i)

In [6]:
log_all = pd.read_csv('driving_log.csv')
for i in os.listdir("./"):
    if i.startswith("driving_log_"):
        log = pd.read_csv(i,header=None)
        print(i, len(log))
        log[0] = log[0].apply(lambda x: x[x.find("IMG") : ])
        log[1] = log[1].apply(lambda x: x[x.find("IMG") : ])
        log[2] = log[2].apply(lambda x: x[x.find("IMG") : ])
        log.columns = ['center', 'left', 'right', 'steering', 'throttle', 'brake', 'speed']
        log_all = pd.concat([log_all, log],ignore_index=True)
print(len(log_all))

driving_log_1.csv 2314
driving_log_2.csv 2646
20019


In [4]:
log_all.head()

,center,left,right,steering,throttle,brake,speed
0,IMG/center_2016_12_01_13_30_48_287.jpg,IMG/left_2016_12_01_13_30_48_287.jpg,IMG/right_2016_12_01_13_30_48_287.jpg,0.0,0.0,0.0,22.148290
1,IMG/center_2016_12_01_13_30_48_404.jpg,IMG/left_2016_12_01_13_30_48_404.jpg,IMG/right_2016_12_01_13_30_48_404.jpg,0.0,0.0,0.0,21.879630
2,IMG/center_2016_12_01_13_31_12_937.jpg,IMG/left_2016_12_01_13_31_12_937.jpg,IMG/right_2016_12_01_13_31_12_937.jpg,0.0,0.0,0.0,1.453011
3,IMG/center_2016_12_01_13_31_13_037.jpg,IMG/left_2016_12_01_13_31_13_037.jpg,IMG/right_2016_12_01_13_31_13_037.jpg,0.0,0.0,0.0,1.438419
4,IMG/center_2016_12_01_13_31_13_177.jpg,IMG/left_2016_12_01_13_31_13_177.jpg,IMG/right_2016_12_01_13_31_13_177.jpg,0.0,0.0,0.0,1.418236


In [7]:
log_all.to_csv('driving_log.csv',index=False)

In [16]:
current_path = 'CarND-Behavioral-Cloning-P3/data/IMG/'
# read in images from center, left and right cameras
img_center = cv2.imread(current_path + log_all["center"].iloc[0].split("/")[-1])

In [ ]:
plt.imshow(img_center)

In [24]:
img_test = img_center / 255.0 - 0.5

In [ ]:
plt.imshow(img_test)

In [3]:
lines = []
with open('driving_log.csv') as file:
    reader = csv.reader(file)
    for line in reader:
        lines.append(line)
        

In [26]:
lines = []
with open('CarND-Behavioral-Cloning-P3/data/driving_log.csv') as file:
    reader = csv.reader(file)
    next(file, None)
    for line in reader:
        lines.append(line)

In [5]:
X_train = np.array(images)
y_train = np.array(measurements)

In [30]:
# set up lambda layer
model = Sequential()
model.add(Lambda(lambda x: (x / 255.0) - 0.5, input_shape=(160,320,3)))
model.add(Cropping2D(cropping=((50, 20), (0, 0))))
model.add(Convolution2D(6,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Convolution2D(16,5,5,activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(1))

In [31]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
lambda_1 (Lambda)                (None, 160, 320, 3)   0           lambda_input_1[0][0]             
____________________________________________________________________________________________________
cropping2d_1 (Cropping2D)        (None, 90, 320, 3)    0           lambda_1[0][0]                   
____________________________________________________________________________________________________
convolution2d_1 (Convolution2D)  (None, 86, 316, 6)    456         cropping2d_1[0][0]               
____________________________________________________________________________________________________
maxpooling2d_1 (MaxPooling2D)    (None, 43, 158, 6)    0           convolution2d_1[0][0]            
___________________________________________________________________________________________

In [8]:
model.compile(loss="mse",optimizer="adam")
model.fit(X_train, y_train, validation_split=0.2,shuffle=True, nb_epoch=1)
model.save('CarND-Behavioral-Cloning-P3/model.h5')
exit()

Train on 6428 samples, validate on 1607 samples
Epoch 1/5
6428/6428 [==============================] - 178s - loss: 46.2662 - val_loss: 0.0205
Epoch 2/5
6428/6428 [==============================] - 175s - loss: 0.0163 - val_loss: 0.0160
Epoch 3/5
6428/6428 [==============================] - 175s - loss: 0.0121 - val_loss: 0.0137
Epoch 4/5
6428/6428 [==============================] - 176s - loss: 0.0103 - val_loss: 0.0129
Epoch 5/5
6428/6428 [==============================] - 175s - loss: 0.0092 - val_loss: 0.0126


In [ ]:
pixel_normalized = pixel / 255

pixel_mean_centered = pixel_normalized - 0.5

In [9]:
list(zip([1,2],[3,4]))

[(1, 3), (2, 4)]

In [17]:
x = [[1],2]
x.extend([3,4])

In [15]:
x.extend([5,6])

In [78]:
x = cv2.imread("what")

In [79]:
x is None

True

In [80]:
x = cv2.imread("./CarND-Behavioral-Cloning-P3/data/IMG/" + "center_2016_12_01_13_30_48_404.jpg")

In [86]:
type(x)

numpy.ndarray